In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [2]:
nQuizz=11
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
# fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/23-Quizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [36]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
# alias

In [5]:
len(alias)

306

In [11]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [12]:
# lQuizz, dfCM

In [13]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Bordelait Lizzie
Garcia_Sara
Soupet-eva
Marylou-Doyen
a-mon a-sal
lohane letourneur
MÉCHAIN-Hermès
raffy margot lacoume justine


# Listage des alias

In [14]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P-N"]=dfCM[u"prénom"].str.lower().str.strip()+"-"+dfCM[u"nom"].str.lower().str.strip()
dfCM["N-P"]=dfCM[u"nom"].str.lower().str.strip()+"-"+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P.N"]=dfCM[u"prénom"].str.lower().str.strip()+"."+dfCM[u"nom"].str.lower().str.strip()
dfCM["N.P"]=dfCM[u"nom"].str.lower().str.strip()+"."+dfCM[u"prénom"].str.lower().str.strip()
# dfCM

In [15]:
pn=dfCM["PN"].tolist()
pnT=dfCM["P-N"].tolist()
pnD=dfCM["P.N"].tolist()
np=dfCM["NP"].tolist()
npT=dfCM["N-P"].tolist()
npD=dfCM["N.P"].tolist()
lNomPrenom={"PN":pn,"P-N":pnT,"P.N":pnD,"NP":np,"N-P":npT,"N.P":npD}

In [16]:
for pseudo in pseudos:
    for nomPrenom in lNomPrenom:
        if unidecode.unidecode(pseudo.lower()) in lNomPrenom[nomPrenom]:
            print pseudo,dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
            alias[pseudo]=[str(dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

Bordelait Lizzie 439788
Soupet-eva 440909
Marylou-Doyen 441677
lohane letourneur 441388
MÉCHAIN-Hermès 437865


In [17]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Garcia_Sara
a-mon a-sal
raffy margot lacoume justine


In [18]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

Garcia_Sara


Nom a chercher:
 garcia


                      PN      id
61  sara garcia--becerra  435581


id :
 435581


a-mon a-sal


...

id :
 438330 438210


raffy margot lacoume justine


Nom a chercher:
 lacoum


                          PN      id
79  justine lacoume-chartier  437894


id :
 437894 441372


In [19]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [20]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [21]:
pseudos

[u'Garcia_Sara', u'a-mon a-sal', u'raffy margot lacoume justine']

In [22]:
# alias["Sofiane"]

In [23]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [24]:
len(alias)

314

In [25]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'APPRIOU-Maurine': ['437791'],
  'Aberard Florelle': ['440340'],
  'Aberard_Florelle': ['440340'],
  'Adam-eulalie': ['435916'],
  'Adriana Mangiarratti': ['442806'],
  'Adriana-Mangiarratti': ['442806'],
  'Agathe raynal': ['436720'],
  'Agatheraynal': ['436720'],
  'Akpinar Neslihan': ['438199'],
  'Amelia Collins': ['437856'],
  'Angelina': ['438425'],
  'Angelina_Boukono': ['438425'],
  'Assimay nahi': ['436153'],
  'Audrey-luu': ['435643'],
  'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
  'BONNEAU MELISSA': ['439214'],
  'BONNEAU Melissa': ['439214'],
  u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
  'Benhamouda-souha': ['442845'],
  'Boileau-Valentine': ['440209'],
  u'Bordelait Lizzie': ['439788'],
  'Bordelait-Lizzie': ['439788'],
  'Boukang': ['438425'],
  'Boy-eleonore': ['441173'],
  'Camille Baldy': ['436384'],
  'Camille Raynaud': ['438388'],
  'Camille raynaud': ['438388'],
  'Camille-Baldy': ['436384'],
  'Carla Rumeau': ['438098'],
  'C

In [26]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [27]:
nbQuestions=[7,7,7,7,7,7,7,6,6,6,6]
nbQ=float(nbQuestions[nQuizz-1])
print nbQ

6.0


In [28]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [29]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,Paulineemeras,4,99.23,[438229],17,438229,None,None
1,Martins Laura,4,109.05,[440840],17,440840,None,None
2,flavie brouxel,4,112.57,[437820],17,437820,None,None
3,Bordelait Lizzie,4,151.31,[439788],17,439788,None,None
4,Aberard_Florelle,4,155.11,[440340],17,440340,None,None
5,laurent-julie,4,162.97,[436286],17,436286,None,None
6,theo-brieux,4,171.23,[436528],17,436528,None,None
7,olaia-yriarte,4,184.17,[437752],17,437752,None,None
8,lea-cremers,4,203.60,[440791],17,440791,None,None
9,Sarah.millepiedtorres,4,211.81,[437850],17,437850,None,None


In [30]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [31]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
0,Paulineemeras,17,99.23,438229
1,Martins Laura,17,109.05,440840
2,flavie brouxel,17,112.57,437820
3,Bordelait Lizzie,17,151.31,439788
4,Aberard_Florelle,17,155.11,440340
5,laurent-julie,17,162.97,436286
6,theo-brieux,17,171.23,436528
7,olaia-yriarte,17,184.17,437752
8,lea-cremers,17,203.60,440791
9,Sarah.millepiedtorres,17,211.81,437850


In [32]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [33]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"].any()<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,Paulineemeras,20,99.23,438229
1,Martins Laura,18,109.05,440840
2,flavie brouxel,17,112.57,437820
3,Bordelait Lizzie,17,151.31,439788
4,Aberard_Florelle,17,155.11,440340
5,laurent-julie,17,162.97,436286
6,theo-brieux,17,171.23,436528
7,olaia-yriarte,17,184.17,437752
8,lea-cremers,17,203.60,440791
9,Sarah.millepiedtorres,17,211.81,437850


In [34]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")

In [35]:
dfNotes

,pseudo,note,temps
id,,,
438229,Paulineemeras,20,99.23
440840,Martins Laura,18,109.05
436286,laurent-julie,17,162.97
439788,Bordelait Lizzie,17,151.31
437850,Sarah.millepiedtorres,17,211.81
437820,flavie brouxel,17,112.57
440209,valentine-boileau,17,230.31
437752,olaia-yriarte,17,184.17
440791,lea-cremers,17,203.60
